In [11]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

In [12]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/A2PM/defensedmodel.csv", mode='a', index=False)




In [3]:
# def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_true))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)
        
#         accuracy = accuracy_score(all_labels, all_preds)

#         precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
#         precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
#         # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
#         # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

#         cm = confusion_matrix(all_labels, all_preds)

#         def calculate_class_metrics_macro(cm, class_index):
#             TP = cm[class_index, class_index]
#             FP = cm[:, class_index].sum() - TP
#             FN = cm[class_index, :].sum() - TP
#             TN = cm.sum() - (TP + FP + FN)
            
#             TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
#             TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
#             FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
#             FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
#             return TPR, TNR, FPR, FNR
            
#         metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
#         TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

#         print(f"Accuracy: {accuracy}")
        
#         print("\nmacro")
#         print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
#         print("\nweighted")
#         print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
#         print()
        
#         print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

#         new_row = {
#             "model" : model_name,
#             "attack_model" : attack_name,
#             'epsilon': eps,
#             'Accuracy': accuracy,
#             'Macro Precision': precision_macro,
#             'Weighted Precision': precision_weighted,
#             'Macro Recall': recall_macro,
#             'Weighted Recall': recall_weighted,
#             'Macro F1': f1_macro,
#             'Weighted F1': f1_weighted,
#             # 'Macro AUC': macro_auc,
#             # 'Weighted AUC': weighted_auc,
#             'TPR': TPR_macro,
#             'FNR': FNR_macro,
#             'TNR': TNR_macro,
#             'FPR': FPR_macro,
#         }
#         new_row_df = pd.DataFrame([new_row])
#         new_row_df.to_csv("/home/jovyan/A2PM/defensedmodel.csv", mode='a', index=False, header=False)




In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [14]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

In [15]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [16]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [17]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Wustl_iiot/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [18]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [20]:
def calculate_performance_metrics(model, X_test, y_test, type):
    metrics_weighted(model, X_test, y_test, type)
    metrics_macro(model, X_test, y_test, type)

def metrics_macro(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    data = {
    "Metric": ["Accuracy", "Macro Precision", "Macro Recall", "Macro F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/defensedmodel.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Macro Precision: {precision * 100:.2f}%")
    print(f"Macro Recall: {recall * 100:.2f}%")
    print(f"Macro F1 Score: {f1 * 100:.2f}%")
    
def metrics_weighted(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)

    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    data = {
    "Metric": ["Accuracy", "Weighted Precision", "Weighted Recall", "Weighted F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/defensedmodel.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Weighted Precision: {precision * 100:.2f}%")
    print(f"Weighted Recall: {recall * 100:.2f}%")
    print(f"Weighted F1 Score: {f1 * 100:.2f}%")

In [21]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(x_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)
        
pattern = (
    {
        "type": "interval",
        "features": numerical_columns,
        #"integer_features": list(range(10, 20)),
        "ratio": 0.1,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 1,
    },
    # {
    #     "type": "combination",
    #     "features": binary_columns,
    #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
    #     "probability": 0.4,
    # },
)

# method = A2PMethod(pattern)

In [24]:
# Define attack methods
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.defences.trainer import AdversarialTrainer, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from a2pm import A2PMethod
from a2pm.callbacks import BaseCallback, MetricCallback, TimeCallback
from a2pm.patterns import BasePattern, CombinationPattern, IntervalPattern
from a2pm.wrappers import BaseWrapper, KerasWrapper, SklearnWrapper, TorchWrapper

attacks = {
    # 'FGSM': FastGradientMethod(estimator=classifier, eps=0.3),
    # 'BIM': BasicIterativeMethod(estimator=classifier, eps=0.3, max_iter=10),
    # 'PGD': ProjectedGradientDescent(estimator=classifier, eps=0.3, max_iter=10),
    # 'DF': DeepFool(classifier=classifier, max_iter=10),
    'A2PM': A2PMethod(pattern),
}

# List to store results
results = []

# Loop through each attack and apply adversarial training with each defense
for attack_name, attack in attacks.items():
    print(f"Applying attack: {attack_name}")

    # Adversarial Training
    # trainer = AdversarialTrainer(classifier, attack, ratio=1.0)
    trainer = Trainer(classifier)

    trainer.fit(x_train, y_train, nb_epochs=5, batch_size=128)

    
    epsilon_values = [0.01, 0.1, 0.2, 0.3]

    for epsilon in epsilon_values:
        X = f'/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{epsilon}_fullattack_X.npy'
        x_test_adv = np.load(X)
        y = f'/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{epsilon}_fullattack_y.npy'
        y_test_adv = np.load(y)
    
        type = f"DNN_defensed_{attack_name}_{epsilon}"
        calculate_performance_metrics(trainer, x_test_adv, y_test, type)




Applying attack: A2PM


TypeError: Can't instantiate abstract class Trainer with abstract method fit

DNN_defensed_FGSM_0.01
Accuracy: 99.98%
Weighted Precision: 99.98%
Weighted Recall: 99.98%
Weighted F1 Score: 99.98%
DNN_defensed_FGSM_0.01
Accuracy: 99.98%
Macro Precision: 99.56%
Macro Recall: 86.22%
Macro F1 Score: 91.43%
DNN_defensed_FGSM_0.1
Accuracy: 99.96%
Weighted Precision: 99.96%
Weighted Recall: 99.96%
Weighted F1 Score: 99.95%
DNN_defensed_FGSM_0.1
Accuracy: 99.96%
Macro Precision: 99.37%
Macro Recall: 85.67%
Macro F1 Score: 91.06%
DNN_defensed_FGSM_0.2
Accuracy: 98.67%
Weighted Precision: 99.17%
Weighted Recall: 98.67%
Weighted F1 Score: 98.88%
DNN_defensed_FGSM_0.2
Accuracy: 98.67%
Macro Precision: 53.50%
Macro Recall: 70.63%
Macro F1 Score: 57.08%
DNN_defensed_FGSM_0.3
Accuracy: 20.34%
Weighted Precision: 98.76%
Weighted Recall: 20.34%
Weighted F1 Score: 31.78%
DNN_defensed_FGSM_0.3
Accuracy: 20.34%
Macro Precision: 41.09%
Macro Recall: 44.92%
Macro F1 Score: 26.70%


In [ ]:
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
# from art.attacks.evasion import (
#     FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent,
#     DeepFool, SaliencyMapMethod, ElasticNet
# )
# from art.estimators.classification import PyTorchClassifier
# from art.defences.trainer import AdversarialTrainer
# import pandas as pd
# import joblib
# import time

# # Load the arrays from the .npz file
# data = np.load('data_arrays.npz')
# x_train = data['X_train']
# x_test = data['X_test']
# x_val = data['X_val']
# y_train = data['y_train']
# y_test = data['y_test']
# y_val = data['y_val']

# X_combined = np.concatenate((x_train, x_test, x_val), axis=0)
# y_combined = np.concatenate((y_train, y_test, y_val), axis=0)

# # Define DNN Model
# class DNNModel(nn.Module):
#     def __init__(self, input_size, output_size):
#         super(DNNModel, self).__init__()
#         self.fc1 = nn.Linear(input_size, 50)
#         self.fc2 = nn.Linear(50, 30)
#         self.fc3 = nn.Linear(30, 20)
#         self.fc4 = nn.Linear(20, output_size)

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = torch.relu(self.fc3(x))
#         x = self.fc4(x)
#         return x

# # Initialize the model
# input_size = X_combined.shape[1]
# output_size = len(np.unique(y_combined))

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = DNNModel(input_size, output_size).to(device)
# model.load_state_dict(torch.load("dnn_pytorch.pt", map_location=device))
# model.eval()

# # Define function to find binary columns
# def find_binary_columns(X_train):
#     binary_columns = []
#     for col in range(X_train.shape[1]):
#         unique_values = np.unique(X_train[:, col])
#         if set(unique_values).issubset({0, 1}):
#             binary_columns.append(col)
#     return binary_columns

# binary_columns = find_binary_columns(x_train)
# numerical_columns = [i for i in range(X_combined.shape[1]) if i not in binary_columns]

# # Define ART classifier
# classifier = PyTorchClassifier(
#     model=model,
#     input_shape=(input_size,),
#     loss=nn.CrossEntropyLoss(),
#     nb_classes=output_size,
#     optimizer=optim.Adam(model.parameters(), lr=0.01),
#     device_type='gpu' if torch.cuda.is_available() else 'cpu'
# )

# # Define attack methods
# attacks = {
#     # 'FGSM': FastGradientMethod(estimator=classifier, eps=0.2),
#     # 'BIM': BasicIterativeMethod(estimator=classifier, eps=0.2, max_iter=10),
#     # 'PGD': ProjectedGradientDescent(estimator=classifier, eps=0.2, max_iter=10),
#     'DF': DeepFool(classifier=classifier, max_iter=10)
# }

# # List to store results
# results = []

# # Loop through each attack and apply adversarial training
# for attack_name, attack in attacks.items():
#     print(f"Applying attack: {attack_name}")

#     # Adversarial Training
#     trainer = AdversarialTrainer(classifier, attack, ratio=1.0)
#     trainer.fit(X_combined, y_combined, nb_epochs=10, batch_size=64)

#     # Generate adversarial examples
#     X_adversarial = attack.generate(x=X_combined)

#     # Convert data to tensor and move to GPU if available
#     X_adversarial_tensor = torch.tensor(X_adversarial, dtype=torch.float32).to(device)
#     y_combined_tensor = torch.tensor(y_combined, dtype=torch.long).to(device)

#     # Predict using the classifier
#     preds_indices = classifier.predict(X_adversarial_tensor)
#     preds_indices_np = preds_indices.argmax(axis=1)

#     # Calculate precision, recall, and F1-score
#     precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
#         y_combined, preds_indices_np, average='micro')
#     precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
#         y_combined, preds_indices_np, average='macro')

#     # Calculate accuracy
#     acc = np.mean(preds_indices_np == y_combined)

#     # Compute the confusion matrix and derived metrics
#     conf_matrix = confusion_matrix(y_combined, preds_indices_np)
#     FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
#     FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
#     TP = np.diag(conf_matrix)
#     TN = conf_matrix.sum() - (FP + FN + TP)

#     TPR = TP / (TP + FN)  # True Positive Rate
#     TNR = TN / (TN + FP)  # True Negative Rate
#     FPR = FP / (FP + TN)  # False Positive Rate
#     FNR = FN / (FN + TP)  # False Negative Rate

#     # Calculate averages
#     TPR_avg = np.mean(TPR)
#     TNR_avg = np.mean(TNR)
#     FPR_avg = np.mean(FPR)
#     FNR_avg = np.mean(FNR)

#     # Prepare a dictionary for the results
#     result_dict = {
#         'Attack': attack_name,
#         'Accuracy': acc,
#         'Micro Precision': precision_micro,
#         'Macro Precision': precision_macro,
#         'Micro Recall': recall_micro,
#         'Macro Recall': recall_macro,
#         'Micro F1 Score': fscore_micro,
#         'Macro F1 Score': fscore_macro,
#         'Average TNR': TNR_avg,
#         'Average FPR': FPR_avg,
#         'Average FNR': FNR_avg,
#         'Macro TNR': TNR_avg,
#         'Macro FNR': FNR_avg,
#         'Macro FPR': FPR_avg
#     }

#     # Add results to the list
#     results.append(result_dict)

# # Convert results to DataFrame and save to CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv('adversarial_trainer_results.csv', index=False)

# print("All results have been saved to adversarial_defenses_results.csv")